<a href="https://colab.research.google.com/github/parthag1201/DocReversalEngine/blob/main/Workflow/WorkflowDiagramV1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install crewai_tools langchain_community langchain_google_genai fpdf2 markdown2

  Using cached google_ai_generativelanguage-0.6.18-py3-none-any.whl.metadata (9.8 kB)
Using cached google_ai_generativelanguage-0.6.18-py3-none-any.whl (1.4 MB)
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [2]:
%pip install crewai agentic-ai

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Install the google-generativeai package
%pip install google-generativeai

  Using cached google_ai_generativelanguage-0.6.15-py3-none-any.whl.metadata (5.7 kB)
Using cached google_ai_generativelanguage-0.6.15-py3-none-any.whl (1.3 MB)
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.18
    Uninstalling google-ai-generativelanguage-0.6.18:
      Successfully uninstalled google-ai-generativelanguage-0.6.18
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-google-genai 2.1.8 requires google-ai-generativelanguage<0.7.0,>=0.6.18, but you have google-ai-generativelanguage 0.6.15 which is incompatible.


In [4]:
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings

c:\Users\akikumar\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# pdf read and write
from fpdf import FPDF
from markdown2 import Markdown

In [6]:
# Warning Control
import warnings
warnings.filterwarnings('ignore')

In [7]:
# Import crewAI library
from crewai import Agent, Task, Crew, Process
from crewai.project import CrewBase, agent, crew, task
from crewai.agents.agent_builder.base_agent import BaseAgent
from typing import List
from crewai.flow.flow import Flow, listen, start

In [38]:
import os

# %pip install python-dotenv

from dotenv import load_dotenv
load_dotenv()

apiKey = os.getenv("GOOGLE_API_KEY3")

# print(apiKey)

In [39]:
#setting up LLM

from crewai import LLM

llm = LLM(model="gemini/gemini-2.5-pro",
                             verbose=True,
                             temperature=0.5,
                             api_key = apiKey)

# print(apiKey)


In [47]:
import os
import re
from tkinter import Tk
from tkinter.filedialog import askdirectory

# Hide the root tkinter window
Tk().withdraw()

# Ask user to select a folder
folder_path = askdirectory(title="Select Folder Containing ABAP Code Files")

abap_code_example = ""
output_html_filename = ""

if folder_path:
    print("Folder selected:", folder_path)

    # List all files in the selected folder
    all_files = os.listdir(folder_path)
    print("\nFiles in the folder:")
    for file in all_files:
        print(" -", file)

    # Read all .txt files and concatenate their content
    for filename in all_files:
        if filename.lower().endswith(".txt"):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'r', encoding='utf-8') as f:
                abap_code_example += f"\n\n// File: {filename}\n" + f.read()

    print("\nCode Preview:\n", abap_code_example[:500])  # Show a preview

    # Create output filename using parent and current folder names
    folder_name = os.path.basename(folder_path.rstrip("/\\"))
    parent_folder = os.path.basename(os.path.dirname(folder_path.rstrip("/\\")))
    combined_name = f"{parent_folder}_{folder_name}"
    output_html_filename = re.sub(r'[\\/*?:"<>|]', "_", combined_name) + ".html"

    print("\nOutput HTML file will be named:", output_html_filename)
else:
    print("No folder selected.")


Folder selected: C:/Users/akikumar/Downloads/OneDrive_2025-07-28/FSTS Gemini/FS and TS/MTD/Code

Files in the folder:
 - 1. ZXQPLU02.txt
 - 2. ZGNMTDSR162489_02.txt

Code Preview:
 

// File: 1. ZXQPLU02.txt
*&---------------------------------------------------------------------*
*&  Include           ZXQPLU02
*&---------------------------------------------------------------------*

 CONSTANTS: lc_exit_cdwn TYPE zgxap_exit_id     VALUE 'SR162489',
*            Business ID
            lc_bus_0420  TYPE zgxap_business_id VALUE 'MTD',
*            Local/Global Indicator
            lc_glid      TYPE zgxap_glid        VALUE 'GT'.

*            Active Flag for User exit activat

Output HTML file will be named: MTD_Code.html


In [48]:


# main.py
import os
from crewai import Agent, Task, Crew, Process
from langchain_google_genai import ChatGoogleGenerativeAI
# from crewai_tools import ScrapeWebsiteTool
# --- Agent Definitions ---

# 1. ABAP Code Interpreter Agent
# This agent is the first point of contact with the raw ABAP code.
# Its primary goal is to understand the code's structure, identify key components
# like FORMs, FUNCTIONs, METHODs, and understand the high-level flow.
abap_interpreter = Agent(
    role='ABAP Code Interpreter',
    goal='Analyze the provided ABAP code to understand its structure, main components (like FORMs, METHODs, FUNCTIONs), and the overall program flow including the includes that each abap code contains.',
    backstory=(
        "As a seasoned ABAP developer with decades of experience, you have an unparalleled ability to read and instantly "
        "comprehend even the most complex and archaic ABAP code. You can see beyond the syntax to the underlying business "
        "logic and program structure. Your task is to distill this complex code into a clear, structured summary that "
        "other agents can use."
        "You look several abap codes as one single entity as one single routine can include another routine and you should be able to understand the whole code as one single entity."
    ),
    verbose=True,
    allow_delegation=False,
    llm=llm
)

# 2. Logic Reviewer Agent
# This agent takes the structured summary from the interpreter and focuses on the
# business logic. It identifies conditional statements (IF/ELSE, CASE), loops (DO, WHILE),
# and subroutine calls to map out the decision points and processes.
logic_reviewer = Agent(
    role='Business Logic Analyst',
    goal='Review the structured summary of the ABAP code to identify and map out the core business logic, including conditional paths, loops, and subroutine calls.',
    backstory=(
        "You are a meticulous business analyst who specializes in reverse-engineering legacy systems. You have a keen eye for "
        "detail and can trace the flow of logic through complex code. Your strength is in identifying the decision points, "
        "data transformations, and processes that define how the program achieves its business objective. You translate "
        "technical flow into logical steps."
        "You focus mainly on business logic and you do not focus on the technicalities of the code."
        "You look at the code as a whole and you do not look at the code as a single routine but you look at the whole code as one web of different routines that are connected to each other. "
    ),
    verbose=True,
    allow_delegation=False,
    llm=llm
)

# 3. Diagram Blocks Generator Agent
# This agent translates the logical steps identified by the reviewer into
# abstract diagram blocks. It decides what should be a process, a decision,
# an input/output, etc., without worrying about the final syntax.
diagram_blocks_generator = Agent(
    role='Workflow Diagram Architect',
    goal='Translate the identified business logic into a list of abstract diagram components (e.g., Start, End, Process, Decision, IO).',
    backstory=(
        "You are a systems architect who thinks visually. You can take a description of a process and immediately see it as a "
        "flowchart. You are an expert in UML and other diagramming methodologies, but your current focus is on defining the "
        "fundamental building blocks of the diagram—the nodes and their types—before they are rendered into a specific format."
    ),
    verbose=True,
    allow_delegation=False,
    llm=llm
)

# 4. Diagram Generation Agent
# This agent takes the abstract blocks and generates the final diagram code
# in a specific format, in this case, MermaidJS.
# mermaid_docs_tool = ScrapeWebsiteTool(website_url='https://mermaid.js.org/intro/')

# Define the Mermaid Code Generator agent
diagram_generator = Agent(
    role='Mermaid Code Generator',
    goal='Generate syntactically correct Mermaid code for various diagrams from a natural language description.',
    backstory=(
        "You are an expert in creating diagrams-as-code using Mermaid. You have a deep understanding of Mermaid's syntax "
        "You know the latest syntax of MermaidJS and its common syntax errors and you have always have been known to give the perfect Mermaid code without any errors especially you should not use parantheses inside the each code block of MemaidJS code. "
        "You also do not make the error of using quote inside the code block of MermaidJS code. "
        "for generating a wide variety of diagrams, including flowcharts, sequence diagrams, Gantt charts, and more. "
        "You can translate natural language descriptions or structured data into clean, readable, and accurate Mermaid code. "
        "You often consult the official Mermaid documentation to ensure you are using the latest and most effective features."
    ),
    # The tool that allows the agent to scrape/read websites
    # tools=[mermaid_docs_tool],
    verbose=True,
    allow_delegation=False,
    llm=llm # Pass your configured LLM instance here
)


# 5. Manager Agent (Quality Control)
# This agent acts as the final quality gate. It reviews the generated diagram
# against the initial code to ensure it's a faithful representation—not too
# detailed (e.g., diagramming every single line) and not too high-level
# (e.g., missing critical logic branches).
manager = Agent(
    role='Quality Assurance Manager',
    goal='Review the final MermaidJS diagram to ensure it accurately represents the original ABAP code\'s logic without being overly complex or too simplistic. Provide the final, approved diagram code.',
    backstory=(
        "With a dual background in software development and project management, you are the ultimate quality gatekeeper. "
        "You ensure that the final product meets the initial requirements. Your job is to look at the generated diagram and the "
        "original code and ask: 'Does this diagram help a developer or analyst understand the code's workflow?' You have the final "
        "say on whether the diagram is approved."
        "You as manager expect the diagram to be a high level diagram with no or minimal technicalities for the end user since its just a addition to FS and should not act as replacement for FS. "
    ),
    verbose=True,
    allow_delegation=True,
    llm=llm
)

## File Handling

In [ ]:
# %pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.8 MB/s eta 0:00:00


In [ ]:
# # 1. Read code from the 'code_files' folder
# import PyPDF2 # Import PyPDF2 here
# code_folder_path = 'code_files'
# if not os.path.exists(code_folder_path):
#     os.makedirs(code_folder_path)
#     # Create a dummy file for demonstration if the folder is empty
#     with open(os.path.join(code_folder_path, 'sample_code.txt'), 'w') as f:
#         f.write("REPORT Z_SAMPLE_REPORT.") # Add some dummy code

# abap_code_example = ""
# for filename in os.listdir(code_folder_path):
#     if filename.endswith(".txt"):
#         with open(os.path.join(code_folder_path, filename), 'r') as f:
#             abap_code_example += f.read() + "\n\n"

# print(abap_code_example)

REPORT Z_SAMPLE_REPORT.




/usr/local/lib/python3.11/dist-packages/PyPDF2/__init__.py:21: DeprecationWarning: PyPDF2 is deprecated. Please move to the pypdf library instead.
  warnings.warn(


In [49]:
# --- Task Definitions ---

# Example ABAP code to be processed.
# This code contains a simple report with a start-of-selection event,
# a loop, a conditional statement, and a form call.

# Task for the Interpreter
interpretation_task = Task(
    description=f"""
    Analyze the following ABAP code. Identify the main program events (like START-OF-SELECTION),
    loops (DO/WHILE), conditional logic (IF/ELSE), and subroutine calls (PERFORM).
    Create a clear, summarized and breif list of these structural components and their sequence.
    You consider all different abap codes for subroutines as a part of a single routine entity.
    Summarize the nested loops to outline the common objective.
    Try to keep the details breif yet concise and summarize the details as much as possible to avoid more technical details.

    ABAP Code:
    ---
    {abap_code_example}
    ---
    """,
    expected_output="A structured text summary outlining the main execution flow, loops, conditions, and subroutine calls in the ABAP code.",
    agent=abap_interpreter
)

# Task for the Logic Reviewer
logic_review_task = Task(
    description="""
    Based on the structured summary of the ABAP code, map out the business logic.
    Focus on the sequence of operations and the decisions being made.
    Extract the logic in a step-by-step manner considering all the subroutines as a part of single routine entity.
    Try to keep the details breif yet concise and summarize the details as much as possible to avoid more technical details, keep it very high-level for the end user.

    """,
    expected_output="A step-by-step description of the program's logic flow, highlighting decision points and repeated processes.",
    agent=logic_reviewer,
    context=[interpretation_task] # This task depends on the output of the interpretation_task
)

# Task for the Diagram Blocks Generator
block_generation_task = Task(
    description="""
    Convert the step-by-step logic flow into a list of abstract diagram blocks.
    Use standard flowchart notation:
    - 'Start' for the beginning
    - 'End' for the end
    - 'Process' for an action (e.g., 'Initialize variables', 'Write message')
    - 'Decision' for a condition (e.g., 'Is counter even?')
    - 'IO' for any input/output operations.
    - 'Loop' for start/end of loops.

    Club multiple Process, Decision, and IO blocks into single blocks where possible to limit the diagram yet showcase a overview.
    Try to reduce the blocks and merge basic blocks of process and decision into single blocks where possible.
    Define the connections between these blocks.
    """,
    # Do show the includes that each a routine has within itself to other subroutines and the connections between them.

    expected_output="A list of objects or a structured text defining each diagram block (e.g., {id: 'A', type: 'Start', label: 'Start Program'}), and the connections between them (e.g., 'A --> B').",
    agent=diagram_blocks_generator,
    context=[logic_review_task]
)

# Task for the Diagram Generator
diagram_generation_task = Task(
    description="""
    Generate the Mermaid code for a flowchart based on a natural language description.
    The diagram should represent a simple user login process, with correct syntax for nodes and connections.
    The flowchart should start with a user entering their credentials.
    The mermaid JS code produced should avoid basic syntax error like using parentheses inside each Mermaid node blocks which needs to avoided as it is found frequently in the output of the previous agents.
    There should be no parantheses inside the mermaid codes of nodes no matter what, it causes a syntax error in the MermaidJS diagram rendering which needs to be avoided at all costs.
    """,
    expected_output="A single code block containing the complete, syntactically correct  Mermaid code for the UML activity diagram.",
    agent=diagram_generator,
    context=[block_generation_task]
)

# Task for the Manager
manager_review_task = Task(
    description=f"""
    Review the generated MermaidJS code. Compare it against the original ABAP code's logic to ensure accuracy and appropriate detail.
    Do check the length and complexity of the diagram as it should neither be too small nor too large or complex but it does not mean that you make it incomplete. The graph should be complete and should not miss any important logic.
    The diagram should clearly show the main loop, the conditional branch (even/odd check), and the subroutine call.
    It should not be cluttered with unnecessary details like variable declarations.
    Also check the final mermaid JS code that it should contain parantheses inside the each code block of MemaidJS code and it should not contain quotes inside the code block of MermaidJS code.
    If the diagram is accurate, provide the final MermaidJS code block as your final answer. If not, provide feedback (though for this run, assume it's correct).
    Do keep in mind that the diagram is a high-level overview and should not include every technical detail.
    The workflow diagram should not act as a replacement for the functional specification (FS) but should be an addition to it.
    Original ABAP Code for reference:
    ---
    {abap_code_example}
    ---
    """,
    expected_output="The final, approved MermaidJS code block, ready for rendering.",
    agent=manager,
    context=[diagram_generation_task]
)

In [50]:
# --- Crew Definition ---

# Assemble the agents and tasks into a crew
abap_to_diagram_crew = Crew(
    agents=[abap_interpreter, logic_reviewer, diagram_blocks_generator, diagram_generator, manager],
    tasks=[interpretation_task, logic_review_task, block_generation_task, diagram_generation_task, manager_review_task],
    process=Process.sequential,  # The tasks will be executed one after another
    verbose= True  # Provides detailed logs of the process
)


# --- Execution ---

if __name__ == "__main__":
    print("🚀 Starting the ABAP to Diagram Crew with Gemini Pro...")
    print("======================================================")
    mermaid_code = abap_to_diagram_crew.kickoff(
    inputs={
        'abap_code': abap_code_example
    }
)

    print("\n========================================")
    print("✅ Crew execution finished!")
    print("\nFinal Result (MermaidJS Code):")
    print("----------------------------------------")
    # The final result is the output of the last task
    print(mermaid_code)
    print("----------------------------------------")

🚀 Starting the ABAP to Diagram Crew with Gemini Pro...


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 37a0a5f6-6e40-4da4-9bc4-a4c1bdd3d2dc                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: ABAP Code Interpreter                                                                                   │
│                                                                                                                 │
│  Task:                                                                                                          │
│      Analyze the following ABAP code. Identify the main program events (like START-OF-SELECTION),               │
│      loops (DO/WHILE), conditional logic (IF/ELSE), and subroutine calls (PERFORM).                             │
│      Create a clear, summarized and breif list of these structural components and their sequence.               │
│      You consider all different abap codes for subroutines as a part of a single routine entity.                │
│      Summarize the nested loops to outline the common objective.                                                │
│      Try to keep the details breif yet concise and summarize the details as much as possible to avoid more      │
│  technical details.                                                                                             │
│                                                                                                                 │
│      ABAP Code:                                                                                                 │
│      ---                                                                                                        │
│                                                                                                                 │
│                                                                                                                 │
│  // File: 1. ZXQPLU02.txt                                                                                       │
│  *&---------------------------------------------------------------------*                                       │
│  *&  Include           ZXQPLU02                                                                                 │
│  *&---------------------------------------------------------------------*                                       │
│                                                                                                                 │
│   CONSTANTS: lc_exit_cdwn TYPE zgxap_exit_id     VALUE 'SR162489',                                              │
│  *            Business ID                                                                                       │
│              lc_bus_0420  TYPE zgxap_business_id VALUE 'MTD',                                                   │
│  *            Local/Global Indicator                                                                            │
│              lc_glid      TYPE zgxap_glid        VALUE 'GT'.                                                    │
│                                                                                                                 │
│  *            Active Flag for User exit activation                                                              │
│   DATA:      l_flg_act      TYPE zgxap_flg_active.                                                              │
│                                                                                                                 │
│  **---Begin of change WO0000000182619 - Return Order IL date in future   "++ANKAUR                              │
│   CONSTANTS: lc_art     TYPE qpart   VALUE '05',                                                                │
│              lc_bwart   TYPE bwart   VALUE '655',                                                               │
│              lc_enddate TYPE char4   VALUE '1231',     

HTTPSConnectionPool(host='telemetry.crewai.com', port=4319): Max retries exceeded with url: /v1/traces (Caused by 
SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get 
local issuer certificate (_ssl.c:1028)')))

 Received None or empty response from LLM call.

...

HTTPSConnectionPool(host='telemetry.crewai.com', port=4319): Max retries exceeded with url: /v1/traces (Caused by 
SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get 
local issuer certificate (_ssl.c:1028)')))

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Quality Assurance Manager                                                                               │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```mermaid                                                                                                     │
│  graph TD;                                                                                                      │
│      A((Inspection Lot Created));                                                                               │
│      B{Is Custom Logic SR162489 Active for MTD Context?};                                                       │
│      C((End No Change));                                                                                        │
│      D{Is Inspection for a Return Order? Type 05 or MvT 655};                                                   │
│      E(Calculate & Set End Date to Dec 31st of Current Year);                                                   │
│      F{Is Action a Final Posting Not a Check?};                                                                 │
│      G(Gather Data & Check Prerequisites for Cool-Down e.g., Insp. Type 08/Z01);                                │
│      H{Is Movement a Goods Receipt Type 101?};                                                                  │
│      I(Calculate End Date: Loop through Cool-Down Rules & Add Duration to Start Date);                          │
│      J(Calculate End Date: Find Latest End Date from Historical Lots or use Today);                             │
│      K(Update Inspection Lot with Final Calculated End Date);                                                   │
│      L((End Lot Updated));                                                                                      │
│                                                                                                                 │
│      A --> B;                                                                                                   │
│      B -- Yes --> D;                                                                                            │
│      B -- No --> C;                                                                                             │
│      D -- Yes --> E;                                                                                            │
│      E --> F;                                                                                                   │
│      D -- No --> F;                                                                                             │
│      F -- Yes --> G;                                                                                            │
│      F -- No --> K;                                                                                             │
│      G --> H;                                                                                                   │
│      H -- Yes --> I;                                                                                            │
│      H -- No --> J;                                                                                             │
│      I --> K;                                                                                                   │
│      J --> K;                                                                                                   │
│      K --> L;                                                                                                   │
│  ```                                                   

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 62542d51-11d5-4265-bb5d-6d1a0c771bed                                                                     │
│  Agent: Quality Assurance Manager                                                                               │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 37a0a5f6-6e40-4da4-9bc4-a4c1bdd3d2dc                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ```mermaid                                                                                       │
│  graph TD;                                                                                                      │
│      A((Inspection Lot Created));                                                                               │
│      B{Is Custom Logic SR162489 Active for MTD Context?};                                                       │
│      C((End No Change));                                                                                        │
│      D{Is Inspection for a Return Order? Type 05 or MvT 655};                                                   │
│      E(Calculate & Set End Date to Dec 31st of Current Year);                                                   │
│      F{Is Action a Final Posting Not a Check?};                                                                 │
│      G(Gather Data & Check Prerequisites for Cool-Down e.g., Insp. Type 08/Z01);                                │
│      H{Is Movement a Goods Receipt Type 101?};                                                                  │
│      I(Calculate End Date: Loop through Cool-Down Rules & Add Duration to Start Date);                          │
│      J(Calculate End Date: Find Latest End Date from Historical Lots or use Today);                             │
│      K(Update Inspection Lot with Final Calculated End Date);                                                   │
│      L((End Lot Updated));                                                                                      │
│                                                                                                                 │
│      A --> B;                                                                                                   │
│      B -- Yes --> D;                                                                                            │
│      B -- No --> C;                                                                                             │
│      D -- Yes --> E;                                                                                            │
│      E --> F;                                                                                                   │
│      D -- No --> F;                                                                                             │
│      F -- Yes --> G;                                                                                            │
│      F -- No --> K;                                                                                             │
│      G --> H;                                                                                                   │
│      H -- Yes --> I;                                                                                            │
│      H -- No --> J;                                                                                             │
│      I --> K;                                                                                                   │
│      J --> K;                                                                                                   │
│      K --> L;                                         


✅ Crew execution finished!

Final Result (MermaidJS Code):
----------------------------------------
```mermaid
graph TD;
    A((Inspection Lot Created));
    B{Is Custom Logic SR162489 Active for MTD Context?};
    C((End No Change));
    D{Is Inspection for a Return Order? Type 05 or MvT 655};
    E(Calculate & Set End Date to Dec 31st of Current Year);
    F{Is Action a Final Posting Not a Check?};
    G(Gather Data & Check Prerequisites for Cool-Down e.g., Insp. Type 08/Z01);
    H{Is Movement a Goods Receipt Type 101?};
    I(Calculate End Date: Loop through Cool-Down Rules & Add Duration to Start Date);
    J(Calculate End Date: Find Latest End Date from Historical Lots or use Today);
    K(Update Inspection Lot with Final Calculated End Date);
    L((End Lot Updated));

    A --> B;
    B -- Yes --> D;
    B -- No --> C;
    D -- Yes --> E;
    E --> F;
    D -- No --> F;
    F -- Yes --> G;
    F -- No --> K;
    G --> H;
    H -- Yes --> I;
    H -- No --> J;
    I --> K;
    

In [51]:
# Dummy placeholder for generated mermaid code
# Replace this with your actual mermaid code logic
def remove_first_and_last_line(text):
    lines = text.splitlines()
    if len(lines) <= 2:
        return ''  # Not enough lines to keep anything
    return '\n'.join(lines[1:-1])

final_mermaid = remove_first_and_last_line(str(mermaid_code))

# Wrap the Mermaid code in a minimal HTML with CDN
html_content = f"""
<!DOCTYPE html>
<html>
<head>
  <meta charset="UTF-8">
  <title>Mermaid Diagram</title>
  <script type="module">
    import mermaid from 'https://cdn.jsdelivr.net/npm/mermaid@10/dist/mermaid.esm.min.mjs';
    mermaid.initialize({{ startOnLoad: true }});
  </script>
</head>
<body>
  <div class="mermaid">
    {final_mermaid}
  </div>
</body>
</html>
"""

# Save the HTML file in current directory
if output_html_filename:
    with open(output_html_filename, 'w', encoding='utf-8') as f:
        f.write(html_content)
    print(f"HTML file saved as: {output_html_filename}")
else:
    print("No output filename found. Make sure Block 1 ran first.")


HTML file saved as: MTD_Code.html


In [52]:
import os
import requests

HCTI_API_USER_ID = os.getenv("HCTI_USER_ID") 
HCTI_API_KEY = os.getenv("HCTI_API_KEY")


# Generate the base filename without extension
image_filename = os.path.splitext(output_html_filename)[0] + ".png"

# Send request to HCTI (assuming credentials and data already set)
response = requests.post(
    url="https://hcti.io/v1/image",
    data={
        'html': html_content  # Use the HTML content generated earlier
    },
    auth=(HCTI_API_USER_ID, HCTI_API_KEY),
    verify=False  # Disable SSL verification for this request (not recommended for production use)
)

if response.status_code == 200:
    image_url = response.json()['url']
    image_data = requests.get(image_url, verify=False).content

    with open(image_filename, "wb") as f:
        f.write(image_data)

    print(f"✅ Image saved as: {image_filename}")
else:
    print("❌ Error:", response.status_code, response.text)


✅ Image saved as: MTD_Code.png


In [23]:
import os
import requests
from tkinter import Tk
from tkinter.filedialog import askopenfilenames

# Set HCTI credentials from environment variables
HCTI_API_USER_ID = os.getenv("HCTI_USER_ID")
HCTI_API_KEY = os.getenv("HCTI_API_KEY")

# Step 1: Open file dialog to choose multiple HTML files
Tk().withdraw()  # Hide the root window
html_paths = askopenfilenames(filetypes=[("HTML files", "*.html")])

if not html_paths:
    print("❌ No files selected.")
    exit()

# Step 2: Loop through selected files
for html_path in html_paths:
    with open(html_path, 'r', encoding='utf-8') as f:
        html_content = f.read()

    image_filename = os.path.splitext(html_path)[0] + ".png"

    # Step 3: Send to HCTI
    response = requests.post(
        url="https://hcti.io/v1/image",
        data={'html': html_content},
        auth=(HCTI_API_USER_ID, HCTI_API_KEY),
        verify=False
    )

    if response.status_code == 200:
        image_url = response.json()['url']
        image_data = requests.get(image_url, verify=False).content

        with open(image_filename, "wb") as f:
            f.write(image_data)

        print(f"✅ Saved: {image_filename}")
    else:
        print(f"❌ Failed for: {html_path} | Error:", response.status_code, response.text)


✅ Saved: C:/Users/akikumar/FSTS/DocReversalEngine/Workflow/Code.png
